# <span style="color:orange"> Final Project: Which Constructor is Able to Produce the Fastest Vehicle? </span>

<font size="4"> 

Nancy Chen, Jessica Sun, Teresa Shi

***

## 1. Introduction

Formula 1, often abbreviated as F1, is the highest class of single-seater auto racing sanctioned by the Fédération Internationale de l'Automobile (FIA). It involves highly specialized, open-wheel racing cars competing on circuits around the world. The sport is known for its cutting-edge technology, high speeds, and skilled drivers, who race in a series of Grands Prix throughout the season to compete for the World Drivers' Championship and the Constructors' Championship.

(question & why relevant & results & comming structure?)

***

## 2. Data Description

For our research question, we will be focusing on datasets `constructors` and `results`. The dataset `constructors` provides a table with 211 rows, each records a constructor's ID, its corresponding name, its nationality, as well as its official URL. The dataset `results` is a comprehensive dataset consisting of 25480 observation and 18 variables that provides the performance of drivers. Each row records the performance of a driver in a specific race, including their point, fastest lap time, etc.

In [43]:
import pandas as pd
import numpy as np
import matplotlib as plt

# import datasets
constructors = pd.read_csv("data_raw/constructors.csv")
results = pd.read_csv("data_raw/results.csv")

In [44]:
print(len(constructors))
print(len(results))

211
25840


### 2.1. Merging Datasets

Since we're interested in paring up the constructor's name (e.g. Toyota) with the actual records of the races, we will merge the `constructors` dataset with `results` based on `constructorId`.

In [53]:
results_const = pd.merge(results[["driverId", "constructorId", "points", "laps", 
                                        "milliseconds", "rank", "fastestLapSpeed"]],
                               constructors[["constructorId", "name"]],
                               on = "constructorId",
                               how = "left").sort_values(by = "constructorId") # sort by constructorId for better examination

results_const.head()

,driverId,constructorId,points,laps,milliseconds,rank,fastestLapSpeed,name
0,1,1,10.0,58,5690616,2,218.300,McLaren
8617,117,1,6.0,69,5992552,\N,\N,McLaren
8647,102,1,9.0,63,6896035,\N,\N,McLaren
8648,117,1,6.0,63,6934748,\N,\N,McLaren
8678,117,1,9.0,80,5857328,\N,\N,McLaren


### 2.2. Cleaning Data

We will exclude the observations with NA values (`\N`, in this dataset) for the convenience of future data manipulation.

In [54]:
results_const.replace(r"\N", np.nan, inplace = True) # replace the "\N" values by nan
cleaned_results_const = results_const.dropna() # drop all nan values
cleaned_results_const = cleaned_results_const.rename(columns = {"name":"constructorName"}) # rename for better understanding

cleaned_results_const.head()

,driverId,constructorId,points,laps,milliseconds,rank,fastestLapSpeed,constructorName
0,1,1,10.0,58,5690616,2,218.300,McLaren
7756,5,1,5.0,57,5771321,7,196.378,McLaren
7778,5,1,3.0,44,5063758,13,232.716,McLaren
7798,5,1,3.0,53,4642399,6,245.036,McLaren
7813,1,1,10.0,61,6966337,2,168.561,McLaren


In order to process the data, we need to transform the data type of columns to numeric and transform the `milliseconds` data to the corresponding time length in hours for better understanding.

In [55]:
print(cleaned_results_const[["milliseconds"]].dtypes) # check data type of `milliseconds`
cleaned_results_const["milliseconds"] = pd.to_numeric(cleaned_results_const["milliseconds"]) # transfer to numeric data type
cleaned_results_const["hours"] = cleaned_results_const["milliseconds"] / 3600000 # transform to hours

print(cleaned_results_const[["fastestLapSpeed"]].dtypes) # check data type of `milliseconds`
cleaned_results_const["fastestLapSpeed"] = pd.to_numeric(cleaned_results_const["fastestLapSpeed"]) # transfer to numeric data type

cleaned_results_const.head()

milliseconds    object
dtype: object
fastestLapSpeed    object
dtype: object


,driverId,constructorId,points,laps,milliseconds,rank,fastestLapSpeed,constructorName,hours
0,1,1,10.0,58,5690616,2,218.300,McLaren,1.580727
7756,5,1,5.0,57,5771321,7,196.378,McLaren,1.603145
7778,5,1,3.0,44,5063758,13,232.716,McLaren,1.406599
7798,5,1,3.0,53,4642399,6,245.036,McLaren,1.289555
7813,1,1,10.0,61,6966337,2,168.561,McLaren,1.935094


### 2.3. Analyse Main Columns

Since we're interested in the difference of speed between vehicles produced by different constructors, we will be mainly focusing on columns including `hours` (finishing time in hours) which indicates the average speed, and `fastestLapSpeed` (fastest lap speed (km/h)) which denotes the best dash speed. Here we will shou the aggregated summary statistics for each of the variables we're working on. 

In [50]:
totalHours_agg = cleaned_results_const.agg(mean_totalHours = ('hours','mean'),
            sd_totalHours =   ('hours','std'),
            min_totalHours =  ('hours','min'),
            max_totalHours =  ('hours','max'),
            count_obs   = ('hours',len))

display(totalHours_agg)

,hours
mean_totalHours,1.639279
sd_totalHours,0.301582
min_totalHours,1.181137
max_totalHours,4.095318
count_obs,3683.000000


In [51]:
fastestSpeed_agg = cleaned_results_const.agg(mean_fastestSpeed = ('fastestLapSpeed','mean'),
            sd_fastestSpeed =   ('fastestLapSpeed','std'),
            min_fastestSpeed =  ('fastestLapSpeed','min'),
            max_fastestSpeed =  ('fastestLapSpeed','max'),
            count_obs   = ('fastestLapSpeed',len))

display(fastestSpeed_agg)

,fastestLapSpeed
mean_fastestSpeed,206.205344
sd_fastestSpeed,20.779461
min_fastestSpeed,148.589000
max_fastestSpeed,257.320000
count_obs,3683.000000


We can also take a look at how many unique constructors are there in our dataset, and examine how many observations, i.e. records of results, exists for each of the constructor.

In [64]:
unique_constructors = pd.unique(cleaned_results_const["constructorName"].sort_values())
print("There are " + str(len(unique_constructors)) + 
      " unique constructors in our cleaned dataset, each of them shown below.")
print(unique_constructors)

cleaned_results_const["constructorName"].value_counts()

There are 30 unique constructors in our cleaned dataset, each of them shown below.
['Alfa Romeo' 'AlphaTauri' 'Alpine F1 Team' 'Aston Martin' 'BAR'
 'BMW Sauber' 'Brawn' 'Caterham' 'Ferrari' 'Force India' 'HRT'
 'Haas F1 Team' 'Honda' 'Jaguar' 'Jordan' 'Lotus' 'Lotus F1'
 'Manor Marussia' 'Marussia' 'McLaren' 'Mercedes' 'Racing Point'
 'Red Bull' 'Renault' 'Sauber' 'Spyker' 'Super Aguri' 'Toro Rosso'
 'Toyota' 'Williams']


Ferrari           581
Red Bull          485
Mercedes          435
McLaren           422
Williams          293
Renault           240
Force India       208
Toro Rosso        175
Sauber            129
BMW Sauber         92
Toyota             91
Lotus F1           79
Haas F1 Team       74
AlphaTauri         57
Alfa Romeo         53
Alpine F1 Team     47
Aston Martin       42
Honda              37
Racing Point       36
BAR                35
Brawn              32
Caterham           12
Marussia           11
Jaguar              7
Manor Marussia      3
Lotus               2
Super Aguri         2
Jordan              1
HRT                 1
Spyker              1
Name: constructorName, dtype: int64

We can see from the table above that Ferrari is the most popular constructor in the race, with 581 records. Red Bull is on the second place, followed by Mercedes and others.

***

## 3. Results

***

## 4. Discussion